1.各種ライブラリ導入

In [9]:
#表示用に使用しています。
from util.functions import trace
import numpy as np

from chainer import functions, optimizers

#cpu計算とgpu計算で使い分けるラッパー
from util.chainer_cpu_wrapper import wrapper

from EncoderDecoderModel import EncoderDecoderModel
import subprocess

ImportError: No module named util.functions

In [2]:
#-------------Explain7 in the Qiita-------------
n_epochs    = 30
n_units     = 625
batchsize   = 100
#学習に使用する文字列の長さ
bprop_len   = 10
#勾配法で使用する敷居値
grad_clip   = 0.5
#学習データの格納場所
data_dir = "data_hands_on"
#モデルの出力場所：checkpoint_dir
checkpoint_dir = "cv"
#-------------Explain7 in the Qiita-------------

In [3]:
def source_to_words(source):
    line = source.replace("¥n", " ").replace("¥t", " ")
    for spacer in ["(", ")", "{", "}", "[", "]", ",", ";", ":", "++", "!", "$", '"', "'"]:
        line = line.replace(spacer, " " + spacer + " ")
    
    words = [w.strip() for w in line.split()]
    return words

In [4]:
def load_data():
    vocab = {}
    print ('%s/angular.js'% data_dir)
    source = open('%s/angular_full_remake.js' % data_dir, 'r').read()
    words = source_to_words(source)
    freq = {}
    dataset = np.ndarray((len(words),), dtype=np.int32)
    for i, word in enumerate(words):
        if word not in vocab:
            vocab[word] = len(vocab)
            freq[word] = 0
        dataset[i] = vocab[word]
        freq[word] += 1

    print('corpus length:', len(words))
    print('vocab size:', len(vocab))
    return dataset, words, vocab, freq

In [5]:
if not os.path.exists(checkpoint_dir):
    os.mkdir(checkpoint_dir)

train_data, words, vocab, freq = load_data()

for f in ["frequent", "rarely"]:
    print("{0} words".format(f))
    print(sorted(freq.items(), key=lambda i: i[1], reverse=True if f == "frequent" else False)[:50])

data_hands_on/angular.js
('corpus length:', 812928)
('vocab size:', 6411)
frequent words
[(')', 71416), ('(', 71416), (';', 50808), ("'", 43648), (',', 43528), ('}', 32528), ('{', 32456), ('=', 27584), ('$', 27024), ('"', 21000), (':', 16448), ('function', 12808), (']', 10616), ('[', 10600), ('var', 10160), ('if', 9808), ('return', 8192), ('!', 5424), ('value', 5296), ('element', 4288), ('0', 4216), ('+', 3888), ('>', 3736), ('scope', 3216), ('name', 3176), ('&&', 3056), ('||', 3024), ('?', 2928), ('i', 2704), ('===', 2544), ('key', 2360), ('else', 2344), ('1', 2256), ('true', 2192), ('expect', 1984), ('name=', 1912), ('==', 1848), ('this.', 1832), ('false', 1544), ('</file>', 1440), ('null', 1440), ('<file', 1440), ('ctrl.', 1280), ('locals', 1040), ('for', 1024), ('/', 1000), ('type=', 992), ('in', 984), ('forEach', 960), ('++', 952)]
rarely words
[('node.parentNode.removeChild', 8), ('user.name=', 8), ('codeName', 8), ('Int32', 8), ('inject.push', 8), ('locale.pluralCat', 8), ('<pre

In [6]:
class CharRNN(FunctionSet):

    """
    ニューラルネットワークを定義している部分です。
    上から順に入力された辞書ベクトル空間を隠れ層のユニット数に変換し、次に隠れ層の入
    力と隠れ層を設定しています。
    同様の処理を2層にも行い、出力層では語彙数に修正して出力しています。
    なお最初に設定するパラメータは-0.08から0.08の間でランダムに設定しています。
    """

    def __init__(self, n_vocab, n_units):
        super(CharRNN, self).__init__(
            embed = F.EmbedID(n_vocab, n_units),
            l1_x = F.Linear(n_units, 4*n_units),
            l1_h = F.Linear(n_units, 4*n_units),
            l2_x = F.Linear(n_units, 4*n_units),
            l2_h = F.Linear(n_units, 4*n_units),
            l3   = F.Linear(n_units, n_vocab),
        )
        for param in self.parameters:
            param[:] = np.random.uniform(-0.08, 0.08, param.shape)

    """
    順伝搬の記述です。
    順伝搬の入力をVariableで定義し、入力と答えを渡しています。
    入力層を先ほど定義したembedを用います。
    隠れ層の入力には、先ほど定義したl1_xを用いて、引数にdropout、隠れ層の状態を渡して
    います。
    lstmに隠れ層第1層の状態とh1_inを渡します。
    2層目も同様に記述し、出力層は状態を渡さずに定義します。
    次回以降の入力に使用するため各状態は保持しています。
    出力されたラベルと答えのラベル比較し、損失を返すのと状態を返しています。
    """

    def forward_one_step(self, x_data, y_data, state, train=True, dropout_ratio=0.5):
        x = Variable(x_data, volatile=not train)
        t = Variable(y_data, volatile=not train)

        h0      = self.embed(x)
        h1_in   = self.l1_x(F.dropout(h0, ratio=dropout_ratio, train=train)) + self.l1_h(state['h1'])
        c1, h1  = F.lstm(state['c1'], h1_in)
        h2_in   = self.l2_x(F.dropout(h1, ratio=dropout_ratio, train=train)) + self.l2_h(state['h2'])
        c2, h2  = F.lstm(state['c2'], h2_in)
        y       = self.l3(F.dropout(h2, ratio=dropout_ratio, train=train))
        state   = {'c1': c1, 'h1': h1, 'c2': c2, 'h2': h2}

        return state, F.softmax_cross_entropy(y, t)

    """
    dropoutの記述を外して予測用のメソッドとして記述しています。
    dropoutにはtrainという引数が存在し、trainの引数をfalseにしておくと動作しない
    ので、予測の時は渡す引数を変えて学習と予測を変えても良いですが、今回は明示的に分る
    ように分けて記述しました。
    """

    def predict(self, x_data, state):
        x = Variable(x_data, volatile=True)

        h0      = self.embed(x)
        h1_in   = self.l1_x(h0) + self.l1_h(state['h1'])
        c1, h1  = F.lstm(state['c1'], h1_in)
        h2_in   = self.l2_x(h1) + self.l2_h(state['h2'])
        c2, h2  = F.lstm(state['c2'], h2_in)
        y       = self.l3(h2)
        state   = {'c1': c1, 'h1': h1, 'c2': c2, 'h2': h2}

        return state, F.softmax(y)

"""
状態の初期化です。
"""
def make_initial_state(n_units, batchsize=100, train=True):
    return {name: Variable(np.zeros((batchsize, n_units), dtype=np.float32),
            volatile=not train)
            for name in ('c1', 'h1', 'c2', 'h2')}

In [7]:
# Prepare RNNLM model
model = CharRNN(len(vocab), n_units)

optimizer = optimizers.RMSprop(lr=2e-3, alpha=0.95, eps=1e-8)
optimizer.setup(model)

学習データのサイズを取得
ジャンプの幅を設定（順次学習しない）
パープレキシティを0で初期化
最初の時間情報を取得
初期状態を現在の状態に付与
状態の初期化
損失を0で初期化

In [8]:
whole_len    = train_data.shape[0]
jump         = whole_len // batchsize
epoch        = 0
start_at     = time.time()
cur_at       = start_at
state        = make_initial_state(n_units, batchsize=batchsize)
accum_loss   = Variable(np.zeros((), dtype=np.float32))
cur_log_perp = np.zeros(())